In [1]:
import warnings
warnings.filterwarnings("ignore")
from gensim.models import KeyedVectors
import numpy as np
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import pickle
import operator

# import natural language toolkit
from gensim.parsing.preprocessing import preprocess_string, strip_tags, strip_punctuation
from nltk.corpus   import stopwords
from nltk.tokenize import word_tokenize

import string

In [2]:
from modules.library.postgresql import PostgresQL
from modules.library.word_models import WordModels
from modules.library.document_similarity import DocumentSimilarity
from modules.library.document_models import DocumentModels

# Accessing the database

Using the module PostgresQL we will load the documents from our database and store them in a list 'documents'.

In [3]:
# connect to the database
pg = PostgresQL() 
pg.connect(database="eurlex_environment_only", user="postgres", password="dbpass")

In [4]:
documents = pg.execute("""
    SELECT * FROM documents;
""")

# Word embeddnig model

In this section we will show how to use the module WordModels to load or train a word embedding model. In order to avoid runnig a time consuming commands, we will comment them out and use pickle instead to store and access already loaded or trained models. 

## Training a word embedding model

In case we want to use pre-trained models to train another model on our data, we can use a method in module WordModels called 'train' in the following way:

In [5]:
texts = ["dogs like chasing cats", "cats like chasing mice", "mice eat cheese", "cheese has holes", "earth flat", "nobody knows working"]

sample_model = WordModels()
sample_model.train(texts, size=300, window=1, min_count=1, epochs=10) # texts is a list of stripped strings

In [6]:
sample_model.embedding.vocab.keys()

dict_keys(['d', 'o', 'g', 's', ' ', 'l', 'i', 'k', 'e', 'c', 'h', 'a', 'n', 't', 'm', 'r', 'f', 'b', 'y', 'w'])

In [14]:
type(sample_model)

modules.library.word_models.WordModels

In [24]:
type(sample_model.embedding)

gensim.models.keyedvectors.FastTextKeyedVectors

## Loading a word embedding model

In case model has already been trained either by the user or has been provided from another source, module WordModels enables us to load it and use it.

In [15]:
wiki_en_path = '../data/fasttext/wiki.en.align.vec'
wiki_en_model = WordModels()
wiki_en_model.load(wiki_en_path)

In [8]:
# with open("wwe.pkl", "wb") as f:
#     pickle.dump(wv_wiki_en, f, protocol=-1)

In [9]:
# with open("wwe.pkl", "rb") as f:
#     wv_wiki_en = pickle.load(f)

In [27]:
type(wiki_en_model.model)

type

## Embedding documents

We have some documents saved in a list 'documents'. Let's embed them using DocumentModels module.

In [10]:
document_texts = [doc['document_text'] for doc in documents]

In [16]:
stop_words = stopwords.words('english') + list(string.punctuation)
document_model = DocumentModels(wiki_en_model.embedding, document_texts, stop_words)

In [20]:
document_model.embed_documents()

In [21]:
type(document_model.embedding)

numpy.ndarray

In [22]:
sum(document_model.embedding[0])

0.392096208051953

In [23]:
type(wiki_en_model.embedding)

gensim.models.keyedvectors.Word2VecKeyedVectors